In [ ]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()
# Set to .../ds-retail

In [ ]:
import ast
import glob
import pandas as pd
import plotly.graph_objects as go
import re

# Getting hyperparameters from the log file

## Batch 1 - 2020/07/10

### Read log file

In [ ]:
hyperparameters_file = "hyperparameters/20200710 - Hyperparameter Search.txt"

with open(hyperparameters_file) as f:
    hp_content = f.readlines()

# you may also want to remove whitespace characters like `\n` at the end of each line
hp_content = [x.strip() for x in hp_content]
hp_content = [x for x in hp_content if x != '']

#hp_content

### Put everything into a DataFrame

In [ ]:
hyperparameters_df = pd.DataFrame()

In [ ]:
def get_hyperparameter(s, hyperparameter, ending):
    pat = r"(?<={}':)(.*)(?={})".format(hyperparameter, ending)
    return re.search(pat, s).group(1).strip()

hyperparameters_df['learning_rate'] = [
    float(get_hyperparameter(x, hyperparameter = 'learning_rate', ending=", 'max_depth'"))
    for x in hp_content]
hyperparameters_df['max_depth'] = [
    int(get_hyperparameter(x, hyperparameter = 'max_depth', ending=", 'num_leaves'"))
    for x in hp_content]
hyperparameters_df['num_leaves'] = [
    int(get_hyperparameter(x, hyperparameter = 'num_leaves', ending=", 'bagging_fraction'"))
    for x in hp_content]
hyperparameters_df['bagging_fraction'] = [
    float(get_hyperparameter(x, hyperparameter = 'bagging_fraction', ending=", 'feature_fraction'"))
    for x in hp_content]
hyperparameters_df['feature_fraction'] = [
    float(get_hyperparameter(x, hyperparameter = 'feature_fraction', ending=", 'min_split_gain'"))
    for x in hp_content]
hyperparameters_df['min_split_gain'] = [
    float(get_hyperparameter(x, hyperparameter = 'min_split_gain', ending=", 'min_child_samples'"))
    for x in hp_content]
hyperparameters_df['min_child_samples'] = [
    int(get_hyperparameter(x, hyperparameter = 'min_child_samples', ending=", 'reg_alpha'"))
    for x in hp_content]
hyperparameters_df['reg_alpha'] = [
    float(get_hyperparameter(x, hyperparameter = 'reg_alpha', ending=", 'reg_lambda'"))
    for x in hp_content]
hyperparameters_df['reg_lambda'] = [
    float(get_hyperparameter(x, hyperparameter = 'reg_lambda', ending="}"))
    for x in hp_content]

#hyperparameters_df

In [ ]:
def get_score(s):
    pat = r"(?<=value: )(.*)(?= with pa)"
    return re.search(pat, s).group(1).strip()

hyperparameters_df['MAE'] = [float(get_score(x)) for x in hp_content]

#hyperparameters_df

In [ ]:
hyperparameters_df['n_trial'] = list(range(1,len(hyperparameters_df)+1))

### Save hyperparameters table - hyperparameters/HP_df.csv

In [ ]:
hyperparameters_df.to_csv('hyperparameters/20200710_HP_df.csv', index=False)

## Batch 2 - 2020/07/13

### Read log file

In [ ]:
hyperparameters_file = "hyperparameters/20200713 - Hyperparameter Search.txt"

with open(hyperparameters_file) as f:
    hp_content = f.readlines()

# you may also want to remove whitespace characters like `\n` at the end of each line
hp_content = [x.strip() for x in hp_content]
hp_content = [x for x in hp_content if x != '']

#hp_content

### Put everything into a DataFrame

In [ ]:
hp_content_params_info = [x for x in hp_content if 'learning_rate' in x]
#hp_content_params_info

In [ ]:
hyperparameters_df = pd.DataFrame()

In [ ]:
def get_hyperparameter(s, hyperparameter, ending):
    pat = r"(?<={}':)(.*)(?={})".format(hyperparameter, ending)
    return re.search(pat, s).group(1).strip()

hyperparameters_df['learning_rate'] = [
    float(get_hyperparameter(x, hyperparameter = 'learning_rate', ending=", 'max_depth'"))
    for x in hp_content_params_info]
hyperparameters_df['max_depth'] = [
    int(get_hyperparameter(x, hyperparameter = 'max_depth', ending=", 'num_leaves'"))
    for x in hp_content_params_info]
hyperparameters_df['num_leaves'] = [
    int(get_hyperparameter(x, hyperparameter = 'num_leaves', ending=", 'bagging_fraction'"))
    for x in hp_content_params_info]
hyperparameters_df['bagging_fraction'] = [
    float(get_hyperparameter(x, hyperparameter = 'bagging_fraction', ending=", 'feature_fraction'"))
    for x in hp_content_params_info]
hyperparameters_df['feature_fraction'] = [
    float(get_hyperparameter(x, hyperparameter = 'feature_fraction', ending=", 'min_child_samples'"))
    for x in hp_content_params_info]
hyperparameters_df['min_child_samples'] = [
    int(get_hyperparameter(x, hyperparameter = 'min_child_samples', ending=", 'reg_alpha'"))
    for x in hp_content_params_info]
hyperparameters_df['reg_alpha'] = [
    float(get_hyperparameter(x, hyperparameter = 'reg_alpha', ending=", 'reg_lambda'"))
    for x in hp_content_params_info]
hyperparameters_df['reg_lambda'] = [
    float(get_hyperparameter(x, hyperparameter = 'reg_lambda', ending="}"))
    for x in hp_content_params_info]

#hyperparameters_df

In [ ]:
hp_content_score_info = hp_content[2::4]
#hp_content_score_info

In [ ]:
hyperparameters_df['MAE'] = [float(x) for x in hp_content_score_info]
#hyperparameters_df

In [ ]:
hp_content_time_info = hp_content[3::4]
#hp_content_time_info

In [ ]:
hyperparameters_df['time_seconds'] = [int(x) for x in hp_content_time_info]
#hyperparameters_df

In [ ]:
hyperparameters_df['n_trial'] = list(range(1,len(hyperparameters_df)+1))

### Save hyperparameters table - hyperparameters/HP_df.csv

In [ ]:
hyperparameters_df.to_csv('hyperparameters/20200713_HP_df.csv', index=False)

# Study

## Get hyperparameters tables

In [ ]:
hyperparameters_df_20200710 = pd.read_csv('hyperparameters/20200710_HP_df.csv')
hyperparameters_df_20200710

In [ ]:
hyperparameters_df_20200713 = pd.read_csv('hyperparameters/20200713_HP_df.csv')
hyperparameters_df_20200713

In [ ]:
hyperparameters_df = pd.concat([hyperparameters_df_20200710, hyperparameters_df_20200713], axis=0)
hyperparameters_df

In [ ]:
hyperparameters_df['n_trial_all'] = list(range(1,len(hyperparameters_df)+1))

In [ ]:
hyperparameters_df_copy = hyperparameters_df.copy()

## Get absolute best parameters

In [ ]:
hyperparameters_df = hyperparameters_df_copy.copy()

In [ ]:
best_hyperparameters = hyperparameters_df.loc[hyperparameters_df['MAE'].idxmin(axis=1)]
best_hyperparameters

## With a groupby (useful for discrete choices)

In [ ]:
hyperparameters_df.groupby('min_child_samples').agg({
    'MAE': ['mean', 'std', 'count']
})

In [ ]:
hyperparameters_df.sort_values('learning_rate', inplace=True)

In [ ]:
hyperparameters_df

## Distribution & Evolution of the hyperparameters

In [ ]:
hyperparameters_df = hyperparameters_df_copy.copy()

In [ ]:
hyperparameters_df = hyperparameters_df[hyperparameters_df['MAE'] < 500]

In [ ]:
hyperparameters_df.sort_values('n_trial_all', inplace=True)

In [ ]:
for variable_to_plot in ['max_depth','num_leaves','bagging_fraction','feature_fraction','min_child_samples','learning_rate','min_split_gain','reg_alpha','reg_lambda']:

    #variable_to_plot = 'min_child_samples'

    import plotly.graph_objects as go

    #hyperparameters_df.sort_values(variable_to_plot, inplace=True)

    # Create traces
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=hyperparameters_df[variable_to_plot],
        y=hyperparameters_df['n_trial_all'],
        mode='markers',
        marker=dict(
            #color=hyperparameters_df['n_trial'], #set color equal to a variable
            #colorscale='Viridis', # one of plotly colorscales
            #colorbar=dict(
            #    title="n_trial"
            #),
            color=hyperparameters_df['MAE'], #set color equal to a variable
            colorscale='Viridis_r', # one of plotly colorscales
            colorbar=dict(
                title="MAE"
            ),
            showscale=True
        ),
        hovertemplate=
        "<b>Trial %{y}</b><br><br>" +
        "Value: %{x:.5f}<br>" +
        "MAE: %{marker.color:.3f}<br>" +
        "<extra></extra>",
        showlegend=False,
        name=variable_to_plot
    ))
    
    if variable_to_plot in ['learning_rate','min_split_gain','reg_alpha','reg_lambda']:
        fig.update_xaxes(type="log")

    
    fig.add_trace(go.Scatter(
        x=[0, max(hyperparameters_df[variable_to_plot])],
        y=[119, 119],
        mode='lines',
        showlegend=False,
        line=dict(color='black', dash='dash')
    ))
    
    fig.add_trace(go.Scatter(
        x=[best_hyperparameters[variable_to_plot], best_hyperparameters[variable_to_plot]],
        y=[min(hyperparameters_df['n_trial_all']), max(hyperparameters_df['n_trial_all'])],
        mode='lines',
        showlegend=False,
        line=dict(color='#e377c2', dash='dash')
    ))

    fig.update_layout(
        title='Evolution of hyperparameter {} by trial'.format(variable_to_plot),
        xaxis_title=variable_to_plot,
        yaxis_title='Number trial (the higher, the latest the trial was)',
        legend={'itemsizing': 'constant'}
    )
    
    fig.update_layout(annotations=[go.layout.Annotation(
            text='Below the line: 1st trial, above, 2nd trial ; only for MAE < 500 ; vertical line: best MAE',
            align='center',
            showarrow=False,
            xref='paper',
            yref='paper',
            x=0.5,
            y=-0.22,
            bordercolor='black',
            borderwidth=1
    )])
    
    fig.show()

## MAE per hyperparameter value

In [ ]:
hyperparameters_df = hyperparameters_df_copy.copy()

In [ ]:
for variable_to_plot in ['max_depth','num_leaves','bagging_fraction','feature_fraction','min_child_samples','learning_rate','min_split_gain','reg_alpha','reg_lambda']:

    #variable_to_plot = 'min_child_samples'

    import plotly.graph_objects as go

    hyperparameters_df.sort_values(variable_to_plot, inplace=True)

    # Create traces
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=hyperparameters_df[variable_to_plot],
        y=hyperparameters_df['MAE'],
        mode='markers', # ['markers', 'lines']
        marker=dict(
            color=hyperparameters_df['n_trial'], #set color equal to a variable
            colorscale='Viridis', # one of plotly colorscales
            colorbar=dict(
                title="Number trial"
            ),
            showscale=True
        ),
        name='MAE'
    ))
    
    if variable_to_plot in ['learning_rate','min_split_gain','reg_alpha','reg_lambda']:
        fig.update_xaxes(type="log")

    fig.update_layout(
        title='Evolution of MAE for different {}'.format(variable_to_plot),
        xaxis_title=variable_to_plot,
        yaxis_title='MAE',
        legend={'itemsizing': 'constant'}
    )

    fig.show()
